In [16]:
from utils import *
setup_lecture()

No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Using transformers v4.11.3
Using datasets v1.16.1


## Entrenando un clasificador de texto

Como vimos al principio del notebook anterios, los modelos como DistilBERT están preentrenados para predecir palabras enmascaradas en una secuencia de texto. Sin embargo, no podemos usar estos modelos de lenguaje directamente para la clasificación de texto; tenemos que modificarlos un poco. Para saber qué modificaciones son necesarias, veamos la arquitectura de un modelo basado en una arquitectura encoder-decoder como DistilBERT.

Primero, el texto es tokenizado y representado como vectores one-hot llamados _token encodings_. El tamaño del vocabulario del tokenizador determina la dimensión de las codificaciones de tokens y, por lo general, son de 20k a 200k tokens únicos. Luego, estas codificaciones de tokens se convierten en _token embeddings_, que son vectores que viven en un espacio de menor dimensión. Los _toke embeddings_ luego se pasan a través de las capas del bloque del encoder para generar un _hidden state_ para cada token de input. Para el objetivo de preentrenamiento del modelado del lenguaje, cada _hidden state_ se envía a una capa que predice los tokens de input enmascarados. Para la tarea de clasificación, reemplazamos la capa de modelado de lenguaje con una capa de clasificación.

> nota: En la práctica, PyTorch omite el paso de crear vectores one-hot para codificaciones de tokens porque multiplicar una matriz con un vector one-hot es lo mismo que seleccionar una columna de la matriz. Esto se puede hacer directamente obteniendo la columna con el ID del token de la matriz. Veremos esto a detalle cuando usemos la clase `nn.Embedding`.

Tenemos dos opciones para entrenar un modelo en nuestro conjunto de datos de Twitter:

- _Feature extraction_: Usamos los hidden states como features y solo entrenamos un clasificador en ellos, sin modificar el modelo preentrenado.

- _Fine-tuning_: Entrenamos todo el modelo de extremo a extremo, lo que también actualiza los parámetros del modelo preentrenado.

En las siguientes secciones exploramos ambas opciones para DistilBERT y vemos sus trade-offs.

### Transformers como extractores de features

Usar un transformer como feature extractor es bastante simple. Primero, congelamos los pesos del cuerpo durante el entrenamiento y usamos los hidden states como features para el clasificador. La ventaja de este enfoque es que podemos entrenar rápidamente un modelo pequeño o poco profundo. Dicho modelo podría ser una capa de clasificación o un método que no dependa de gradientes, como un random forrest. Este método es especialmente conveniente si no tenemos GPUs, ya que los hidden states solo deben calcularse una vez.

#### Usando modelos preentrenados

Vamos a usar otra _auto class_ conveniente de Transformers llamada `AutoModel`. Similar a la clase `AutoTokenizer`, `AutoModel` tiene un método `from_pretrained()` para cargar los pesos de un modelo que ya ha sido entrenado. Usemos este método para cargar el checkpoint de DistilBERT:

In [17]:
from transformers import AutoModel

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

Aquí usamos PyTorch para verificar si hay un GPU disponible o no, y luego encadenamos el método `nn.Module.to()` de PyTorch al cargador de modelos. Esto asegura que el modelo se ejecutará en la GPU si tenemos una. De lo contrario, el modelo se ejecutará en la CPU, que puede ser más lenta.



La clase `AutoModel` convierte los _token encodings_ en _token embeddings_ y luego las pasa al encoder stack para devolver los _hidden states_. Veamos cómo extraer estos estados de nuestro corpus.

### Apartado: Interoperabilidad entre frameworks

Aunque el código que vamos a usar está principalmente en PyTorch, Transformers tiene buena interoperabilidad con TensorFlow y JAX. Esto significa que solo necesitan cambiar un par líneas de código para cargar un modelo preentrenado en su framework preferido! Por ejemplo, podemos cargar DistilBERT en TensorFlow usando la clase `TFAutoModel`:

In [18]:
from transformers import TFAutoModel

tf_model = TFAutoModel.from_pretrained(model_ckpt)

Esta interoperabilidad es especialmente útil cuando un modelo solo se publica en un framework, pero nos gustaría usarlo en otro. Por ejemplo, el [modelo XLM-RoBERTa](https://huggingface.co/xlm-roberta-base) que vamos a usar para NER mas adelante, solo tiene pesos de PyTorch, por lo que si intentamos cargarlo en TensorFlow como hicimos antes:

```python
tf_xlmr = TFAutoModel.from_pretrained("xlm-roberta-base")
```

Nos va a dar un error. En estos casos, podemos especificar el argumento `from_pt=True` en el metodo `TfAutoModel.from_pretrained()`, y la libreria automaticamente va a bajar y convertir los pesos de PyTorch por uno:

In [19]:
tf_xlmr = TFAutoModel.from_pretrained("xlm-roberta-base", from_pt=True)

Como puedem ver, es facil cambiar entre frameworks en Transformers. En la mayoría de los casos, solo pueden agregar un prefijo "TF" a las clases y van a tener las clases equivalentes de TensorFlow 2.0. Cuando usamos el string `"pt"` (por ejemplo, en la siguiente sección), que es la abreviatura de PyTorch, simplemente reemplácenla con "`tf"`, que es la abreviatura de TensorFlow.

### Fin de apartado

#### Extrayendo los ultimos _hidden states_

Regresemos al ejemplo y recuperemos los últimos hidden states de un solo string. Lo primero que debemos hacer es codificar el string y convertir los tokens en tensores PyTorch. Esto se puede hacer proporcionando el argumento `return_tensors="pt"` al tokenizador de la siguiente manera:

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
print(f"Input tensor shape: {inputs['input_ids'].size()}")

Input tensor shape: torch.Size([1, 6])


Como podemos ver, el tensor resultante tiene la forma `[batch_size, n_tokens]`. Ahora que tenemos las encodings como tensor, el paso final es colocarlas en el mismo dispositivo que el modelo y pasar los inputs de la siguiente manera:

In [21]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1565, -0.1862,  0.0528,  ...,
-0.1188,  0.0662,  0.5470],
         [-0.3575, -0.6484, -0.0618,  ..., -0.3040,  0.3508,  0.5221],
         [-0.2772, -0.4459,  0.1818,  ..., -0.0948, -0.0076,  0.9958],
         [-0.2841, -0.3917,  0.3753,  ..., -0.2151, -0.1173,  1.0526],
         [ 0.2661, -0.5094, -0.3180,  ..., -0.4203,  0.0144, -0.2149],
         [ 0.9441,  0.0112, -0.4714,  ...,  0.1439, -0.7288, -0.1619]]]),
hidden_states=None, attentions=None)


Aquí usamos el context manager `torch.no_grad()` para deshabilitar el cálculo automático de la gradiente. Esto es útil para el paso de inferencia ya que reduce la memoria de los cálculos. Según la configuración del modelo, el output puede contener varios objetos, como hidden states, losses o attentions, organizados en una clase similar a una `namedtuple` en Python. En nuestro ejemplo, el output del modelo es una instancia de `BaseModelOutput`, y podemos acceder a sus atributos por nombre. El modelo actual devuelve solo un atributo, que es el último hidden state, así que veamos su forma:

In [22]:
outputs.last_hidden_state.size()

torch.Size([1, 6, 768])

Viendo el tensor de hidden state, vemos que tiene la forma `[batch_size, n_tokens, hidden_dim]`. En otras palabras, devuelve un vector de 768 dimensiones para cada uno de los 6 tokens de input. Para las tareas de clasificación, es una práctica común usar simplemente el hidden state asociado con el token `[CLS]` como input feature. Dado que este token aparece al comienzo de cada secuencia, podemos extraerlo simplemente indexándolo en `outputs.last_hidden_state`:

In [23]:
outputs.last_hidden_state[:,0].size()

torch.Size([1, 768])

Ahora que sabemos cómo sacar el último _hidden state_ para un solo string, hagamos lo mismo para todo el dataset creando una nueva columna `hidden_state` que almacene todos estos vectores. Como hicimos con el tokenizador, usamos el método `map()` de `DatasetDict` para extraer todos los hidden states de una vez. Lo primero que debemos hacer es meter los pasos anteriores en una función de procesamiento:

In [24]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

La única diferencia entre esta función y nuestra lógica anterior es el paso final en el que colocamos el hidden state final otra vez en el CPU como una matriz NumPy. El método `map()` requiere que la función de procesamiento devuelva objetos Python o NumPy cuando usamos batched inputs.

Dado que nuestro modelo espera tensores como inputs, lo siguiente que tenemos que hacer es convertir las columnas `input_ids` y `attention_mask` al formato `"torch"`:

In [25]:
from datasets import load_dataset
emotions = load_dataset("dair-ai/emotion")

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)
emotions_encoded.set_format("torch", 
                            columns=["input_ids", "attention_mask", "label"])

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Luego podemos extraer el hidden state a traves de todos los splits en una sola pasada:

In [26]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

AttributeError: 'list' object has no attribute 'to'

Tomen en cuenta que no configuramos `batch_size=None` en este caso, porque en su lugar se usa el valor default `batch_size=1000`. Aplicar la función `extract_hidden_states()` ha agregado una nueva columna `hidden_state` a nuestro dataset:

In [ ]:
emotions_hidden["train"].column_names

Ahora que tenemos los hidden states asociados con cada tweet, el siguiente paso es entrenar un clasificador sobre ellos. Para hacer eso, vamos a necesitar un feature matrix:

#### Creando un feature matrix

El dataset preprocesado ahora tiene toda la información que necesitamos para entrenar un clasificador. Vamos a usar los hidden stattes como input features y las etiquetas como targets. Podemos crear fácilmente las matrices correspondientes en el formato Scikit-Learn asi:

In [ ]:
import numpy as np

X_train = np.array(emotions_hidden["train"]["hidden_state"])
X_valid = np.array(emotions_hidden["validation"]["hidden_state"])
y_train = np.array(emotions_hidden["train"]["label"])
y_valid = np.array(emotions_hidden["validation"]["label"])
X_train.shape, X_valid.shape

Antes de entrenar un modelo en los hidden states, es una buena práctica hacer un sanity check para garantizar que proveen una representación útil de las emociones que queremos clasificar.

#### Visualizar el training set

Dado que visualizar los estados ocultos en 768 dimensiones es complicado, usaremos el algoritmo UMAP[1] para proyectar los vectores en 2D. Dado que UMAP funciona mejor cuando los features se escalan para estar en el intervalo [0,1], primero aplicaremos un `MinMaxScaler` y luego usaremos la implementación de UMAP de la libreria `umap-learn` para reducir los hidden states:

L. McInnes, J. Healy, and J. Melville, ["UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction"](https://arxiv.org/abs/1802.03426), (2018).

In [ ]:
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler

# Scale features to [0,1] range
X_scaled = MinMaxScaler().fit_transform(X_train)
# Initialize and fit UMAP
mapper = UMAP(n_components=2, metric="cosine").fit(X_scaled)
# Create a DataFrame of 2D embeddings
df_emb = pd.DataFrame(mapper.embedding_, columns=["X", "Y"])
df_emb["label"] = y_train
df_emb.head()

El resultado es una matriz con la misma cantidad de muestras de entrenamiento, pero con solo 2 features en lugar de las 768 con las que comenzamos. Investiguemos los datos comprimidos un poco más y tracemos la densidad de puntos para cada categoría por separado:

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(7,5))
axes = axes.flatten()
cmaps = ["Greys", "Blues", "Oranges", "Reds", "Purples", "Greens"]
labels = emotions["train"].features["label"].names

for i, (label, cmap) in enumerate(zip(labels, cmaps)):
    df_emb_sub = df_emb.query(f"label == {i}")
    axes[i].hexbin(df_emb_sub["X"], df_emb_sub["Y"], cmap=cmap,
                   gridsize=20, linewidths=(0,))
    axes[i].set_title(label)
    axes[i].set_xticks([]), axes[i].set_yticks([])

plt.tight_layout()
plt.show()

>nota: Estas son solo proyecciones a un espacio de menor dimensión. El hecho de que algunas categorías se superpongan no significa que no sean separables en el espacio original. Por otro lado, si son separables en el espacio proyectado, van a serlo en el espacio original.

A partir de este plot podemos ver algunos patrones claros: los sentimientos negativos como "sadness", "anger" y "fear" ocupan regiones similares con distribuciones ligeramente diferentes. Por otro lado, la 'joy' y el 'love' están bien separados de las emociones negativas y también comparten un espacio similar. Finalmente, "surprise" está esparcida por todo el lugar. Aunque es posible que hayamos esperado cierta separación, esto no está garantizado de ninguna manera ya que el modelo no fue entrenado para saber la diferencia entre estas emociones. Solo los aprendió implícitamente al adivinar las palabras enmascaradas en los textos.

#### Entrenando un clasificador simple


Hemos visto que los estados ocultos son algo diferentes entre las emociones, aunque para varias de ellas no existe un límite evidente. Usemos estos estados ocultos para entrenar un modelo de regresión logística con Scikit-Learn. El entrenamiento de un modelo tan simple es rápido y no necesita una GPU:



In [ ]:
# We increase `max_iter` to guarantee convergence 
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(X_train, y_train)

In [ ]:
lr_clf.score(X_valid, y_valid)

En cuanto a la precisión, puede parecer que nuestro modelo es solo un poco mejor que el aleatorio, pero dado que estamos tratando con un conjunto de datos multiclase desequilibrado, en realidad es significativamente mejor. Podemos examinar si nuestro modelo es bueno comparándolo con un baseline simple. En Scikit-Learn hay un `DummyClassifier` que se puede usar para construir un clasificador con heurísticas simples, como elegir siempre la clase mayoritaria o dibujar siempre una clase aleatoria. En este caso, la heurística de mejor rendimiento es elegir siempre la clase más frecuente, lo que tira una precisión de alrededor del 35%:

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_valid, y_valid)

Por lo tanto, nuestro clasificador simple con embeddings de DistilBERT es significativamente mejor que nuestra baseline. Podemos investigar más a fondo el rendimiento del modelo mirando la matriz de confusión del clasificador, que nos dice la relación entre las etiquetas verdaderas y las predecidas:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()
    
y_preds = lr_clf.predict(X_valid)
plot_confusion_matrix(y_preds, y_valid, labels)

Podemos ver que "anger" y "fear" se confunden con mayor frecuencia con "sadness", lo que concuerda con la observación que hicimos al visualizar los embeddings. Además, 'love' y 'surprise' se confunden con frecuencia con 'joy'.

En la siguiente parte, vamos a ver el enfoque de fine-tunning, que nos da un rendimiento de clasificación superior. Sin embargo, es importante tener en cuenta que hacer esto requiere más recursos, como GPUs, que podrían no estar disponibles en su empresa. En casos como estos, un enfoque basado en features puede ser un buen compromiso entre ML tradicional y DL.

### Fine-Tuning Transformers

Exploremos ahora lo que se necesita para ajustar un transformer end-to-end. Con el enfoque de fine-tuning, no usamos los estados ocultos como features fijos, sino que los entrenamos. Esto requiere que la cabeza de clasificacion sea diferenciable, por esto, este método suele utilizar una red neuronal para la clasificación.

Entrenar los estados ocultos que sirven como inputs para el modelo de clasificación nos ayuda a evitar el problema de trabajar con datos que pueden no ser adecuados para la tarea de clasificación. En cambio, los estados ocultos iniciales se adaptan durante el entrenamiento para disminuir la pérdida del modelo y así aumentar su rendimiento.

Usaremos la API `Trainer` de Transformers para simplificar el training loop. 

#### Cargando un modelo preentrenado

Lo primero que necesitamos es un modelo DistilBERT preentrenado. La única pequeña modificación es que usamos el modelo `AutoModelForSequenceClassification` en lugar de `AutoModel`. La diferencia es que el modelo `AutoModelForSequenceClassification` tiene una cabeza de clasificacion sobre los outputs del modelo preentrenado, que se pueden entrenar fácilmente con el modelo base. Solo necesitamos especificar cuántas etiquetas tiene que predecir el modelo (seis en nuestro caso), ya que esto dicta la cantidad de outputs que tiene la cabeza de clasificacion:

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 6
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

Vamos a ver una advertencia de que algunas partes del modelo se inicializan aleatoriamente. Esto es normal ya que la cabeza de clasificacion no ha sido entrenada. El siguiente paso es definir las métricas que usaremos para evaluar el rendimiento de nuestro modelo durante el ajuste.

#### Definiendo las metricas de rendimiento

Para monitorear las métricas durante el entrenamiento, necesitamos definir una función `compute_metrics()` para el `Trainer`. Esta función recibe un objeto `EvalPrediction` (que es un named tuple con atributos `predictions` y `label_ids`) y necesita devolver un diccionario que asigna el nombre de cada métrica a su valor. Para nuestra aplicación, calcularemos la puntuación de $F_1$ y la precisión del modelo:



In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

Con el dataset y las métricas listos, solo tenemos dos cosas finales antes de definir la clase `Trainer`:

1. Iniciar sesión en Hugging Face Hub. Esto nos permitirá cargar nuestro modelo a nuestra cuenta en Hub y compartirlo.
2. Definir todos los hiperparámetros para el training run.

#### Entrenando el modelo

Para hacer login desde Colab, pueden usar esta funcion:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Esto mostrará un widget en el que puede ingresar su usuario y contraseña, o un token de acceso con privilegios de escritura. Pueden encontrar detalles sobre cómo crear tokens de acceso en la [documentación de Hub](https://huggingface.co/docs/hub/security#user-access-tokens). Si están trabajando en la terminal, pueden iniciar sesión ejecutando el siguiente comando:

```bash
$ huggingface-cli login
```

Para definir los parámetros de entrenamiento, usamos la clase `TrainingArguments`. Esta clase almacena mucha información y nos da control detallado sobre el entrenamiento y la evaluación. El argumento más importante para especificar es `output_dir`, que es donde se almacenan todos los artefactos del entrenamiento. Aquí hay un ejemplo de `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True, 
                                  log_level="error")

Aquí también establecemos el tamaño del batch, learning rate y el número de épocas, y especificamos cargar el mejor modelo al final del training run. Con esto, podemos instanciar y afinar nuestro modelo con el `Trainer`:

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train();

Si observamos los logs, podemos ver que nuestro modelo tiene una puntuación de $F_1$ en el dataset de validación de alrededor de 92%. Esta es una mejora significativa con respecto al enfoque basado en features.

Podemos echar un vistazo más detallado a las métricas de entrenamiento calculando la matriz de confusión. Para visualizar la matriz de confusión, primero debemos obtener las predicciones en el conjunto de validación. El método `predict()` de la clase `Trainer` devuelve varios objetos útiles que podemos usar para la evaluación:

In [ ]:
# hide_output
preds_output = trainer.predict(emotions_encoded["validation"])

El resultado del método `predict()` es un objeto `PredictionOutput` que contiene matrices de `predictions` y `label_ids`, junto con las métricas que le pasamos al trainer. Por ejemplo, se puede acceder a las métricas en el dataset de validación asi:

In [ ]:
preds_output.metrics

También tiene las predicciones sin procesar para cada clase. Podemos decodificar las predicciones usando `np.argmax()`. Esto produce las etiquetas predecidas y tiene el mismo formato que las etiquetas devueltas por los modelos de Scikit-Learn en el enfoque basado en features:

In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)

Con las predicciones podemos generar la matriz de confusion:

In [ ]:
plot_confusion_matrix(y_preds, y_valid, labels)

Esto está mucho más cerca de la matriz de confusión diagonal ideal. La categoría 'love' todavía se confunde a menudo con 'joy', lo que parece natural. "surprise" también se confunde con frecuencia con "joy", o se confunde con "fear". En general, el rendimiento del modelo parece bastante bueno, pero antes de darlo por terminado, profundicemos un poco más en los tipos de errores que es probable que cometa nuestro modelo.

#### Analisis de errores

Antes de continuar, debemos investigar un poco más las predicciones de nuestro modelo. Una técnica simple pero poderosa es ordenar las muestras de validación por el loss del modelo. Cuando pasamos la etiqueta durante el pase hacia adelante, el loss se calcula y se devuelve automáticamente. Aquí hay una función que devuelve el loss junto con la etiqueta predecida:

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # Place all input tensors on the same device as the model
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device), 
                             reduction="none")

    # Place outputs on CPU for compatibility with other dataset columns   
    return {"loss": loss.cpu().numpy(), 
            "predicted_label": pred_label.cpu().numpy()}

Usando `map()` otra vez, podemos aplicar esta funcion para obtener el loss de todas las muestras:

In [ ]:
# Convert our dataset back to PyTorch tensors
emotions_encoded.set_format("torch", 
                            columns=["input_ids", "attention_mask", "label"])
# Compute loss values
emotions_encoded["validation"] = emotions_encoded["validation"].map(
    forward_pass_with_label, batched=True, batch_size=16)

Finalmente, creamos un `DataFrame` con los textos, losses y etiquetas predecidas/reales:

In [ ]:
emotions_encoded.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = emotions_encoded["validation"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"]
                              .apply(label_int2str))

Ahora podemos ordenar fácilmente `emotions_encoded` por las pérdidas en orden ascendente o descendente. El objetivo de este ejercicio es detectar uno de los siguientes:

- _Etiquetas incorrectas_: todos los procesos que agregan etiquetas a los datos pueden tener fallas. Los anotadores pueden cometer errores o no estar de acuerdo, mientras que las etiquetas que se infieren de otros features pueden estar equivocadas. Si fuera fácil anotar datos automáticamente, entonces no necesitaríamos un modelo para hacerlo. Por lo tanto, es normal que existan algunos ejemplos mal etiquetados. Con este enfoque, podemos encontrarlos y corregirlos rápidamente.

- _Características del dataset_: Los datasets en el mundo real siempre son un poco desordenados. Cuando se trabaja con texto, los caracteres especiales o strings en los inputs pueden tener un gran impacto en las predicciones del modelo. Inspeccionar las predicciones más débiles del modelo puede ayudar a identificar dichas características, y limpiar los datos o inyectar ejemplos similares puede hacer que el modelo sea más sólido.

Primero veamos las muestras de datos con las mayores loss:


In [ ]:
df_test.sort_values("loss", ascending=False).head(10)

Podemos ver claramente que el modelo predijo incorrectamente algunas de las etiquetas. Por otro lado, parece que hay bastantes ejemplos sin una clase clara, que podrían estar mal etiquetados o requerir una nueva clase por completo. En particular, "joy" parece estar mal etiquetado varias veces. Con esta información podemos refinar el conjunto de datos, lo que a menudo puede conducir a una ganancia de rendimiento tan grande (o más) como tener más datos o modelos más grandes.

Al observar las muestras con el loss más bajo, observamos que el modelo parece tener más confianza al predecir la clase de "sadness". Los modelos de DL son muy buenos para encontrar y explotar shortcuts para llegar a una predicción. Por esta razón, también vale la pena invertir tiempo en ver los ejemplos en los que el modelo tiene más confianza, para que podamos estar seguros de que el modelo no explota indebidamente ciertas características del texto. Entonces, veamos también las predicciones con el loss más pequeño:

In [ ]:
df_test.sort_values("loss", ascending=True).head(10)

Ahora sabemos que "joy" a veces está mal etiquetada y que el modelo confía más en predecir la etiqueta "sadness". Con esta información, podemos realizar mejoras específicas en nuestro dataset y también vigilar la clase en la que el modelo parece tener mucha confianza.

El último paso antes de servir el modelo entrenado es guardarlo para su uso posterior. Transformers nos permite hacer esto en unn par de pasos.

#### Guardando y compartiendo el modelo

La comunidad de NLP se beneficia bastante al compartir modelos previamente entrenados y ajustados, y todos pueden compartir sus modelos con otros usando Hugging Face Hub. Cualquier modelo generado por la comunidad se puede descargar del Hub tal como descargamos el modelo DistilBERT. Con la API `Trainer`, guardar y compartir un modelo es facil:

In [ ]:
trainer.push_to_hub(commit_message="Training completed!")

También podemos usar el modelo ajustado para hacer predicciones sobre nuevos tweets. Como hemos enviado nuestro modelo al Hub, ahora podemos usarlo con la función `pipeline()`, tal como lo hicimos en el notebook de intro. Primero, carguemos el pipeline:

In [ ]:
from transformers import pipeline

# Change `my_user` to your Hub username
model_id = "MY_USER/distilbert-base-uncased-finetuned-emotion"
classifier = pipeline("text-classification", model=model_id)

Ahora probemos el pipeline con un tweet de muestra:

In [ ]:
custom_tweet = "I saw a movie today and it was really good."
preds = classifier(custom_tweet, return_all_scores=True)

Finalmente, podemos plotear la probabilidad de cada clase en un gráfico de barras. Claramente, el modelo estima que la clase más probable es "joy", lo que parece razonable dado el tuit:

In [ ]:
preds_df = pd.DataFrame(preds[0])
plt.bar(labels, 100 * preds_df["score"], color='C0')
plt.title(f'"{custom_tweet}"')
plt.ylabel("Class probability (%)")
plt.show()

#### Fine-tunning con Keras

Si usan TensorFlow, también es posible ajustar sus modelos con la API de Keras. La principal diferencia con la API de PyTorch es que no hay una clase de `Trainer`, ya que los modelos de Keras ya proporcionan un método `fit()` integrado. Para ver cómo funciona esto, primero carguemos DistilBERT como un modelo de TensorFlow

In [ ]:
from transformers import TFAutoModelForSequenceClassification

tf_model = (TFAutoModelForSequenceClassification
            .from_pretrained(model_ckpt, num_labels=num_labels))

Luego, vamos a convertir nuestros datasets al formato `tf.data.Dataset`. Como ya aplicamos padding a los inputs tokenizados, podemos hacerlo fácilmente aplicando el método `to_tf_dataset()` a las emociones codificadas:

In [ ]:
# The column names to convert to TensorFlow tensors
tokenizer_columns = tokenizer.model_input_names

tf_train_dataset = emotions_encoded["train"].to_tf_dataset(
    columns=tokenizer_columns, label_cols=["label"], shuffle=True,
    batch_size=batch_size)
tf_eval_dataset = emotions_encoded["validation"].to_tf_dataset(
    columns=tokenizer_columns, label_cols=["label"], shuffle=False,
    batch_size=batch_size)

Aquí también hicimos shuffle del dataset de entrenamiento y definimos el tamaño del batch para este dataset y el de validacion. Lo último que tenemos que hacer es compilar y entrenar el modelo:

In [ ]:
import tensorflow as tf

tf_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy())

tf_model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=2)

`model.compile()` es el unico paso adicional que expone el API de Keras a diferencia del API de Scikit-learn. En el metodo `compile()` especificamos el optimizador, la funcion de costo y las metricas a tomar en cuenta.

## Conclusion

Ahora saben cómo entrenar un transformer para clasificar las emociones en los tweets. Hemos visto dos enfoques complementarios basados ​​en features y fine-tuning, e investigamos sus fortalezas y debilidades.

Sin embargo, este es solo el primer paso en la creación de una aplicación del mundo real con transformers. Aquí hay una lista de desafíos que es probable que experimenten en NLP:

Mi jefe quiere mi modelo en producción ayer:
En la mayoría de las aplicaciones, nuestro modelo no solo se sienta en algún notebook, sino que queremos asegurarnos de que esté sirviendo predicciones. Cuando enviamos un modelo al Hub, se crea automáticamente un endpoihnt de inferencia al que se puede llamar con HTTP requests. Vean la [documentación](https://api-inference.huggingface.co/docs/python/html/index.html) de la API de inferencia para saber mas.

¡Mis usuarios quieren predicciones más rápidas!:
Ya hemos visto un enfoque para este problema: usar DistilBERT. Mas adelante vamos a ver _knowledge distillation_ (el proceso mediante el cual se creó DistilBERT), junto con otros trucos para acelerar sus transformers.


¿Tu modelo también puede hacer X?:
Como hemos mencionado en la clase, los transformers son extremadamente versátiles. En el resto de clases, vamos a explorar una variedad de tareas, como question answering y named entity recognition, todas usando la misma arquitectura básica.

¡Ninguno de mis textos está en inglés!:
Los transformers también vienen en una variedad multilingüe, y los vamos a usar para NER para abordar varios idiomas a la vez.

¡No tengo ninguna etiqueta!:
Si hay muy pocos datos etiquetados disponibles, es posible que el fine-tuning no sea una opción. Mas adelante, vamos a ver algunas técnicas para lidiar con esta situación.

Ahora que hemos visto lo que implica entrenar y compartir un transformer, en las proximas clases vamos a ver la implementación de nuestro propio modelo transformer desde cero.